In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

train_dt = pd.read_csv("/kaggle/input/titanic/train.csv")
test_dt = pd.read_csv("/kaggle/input/titanic/test.csv")

In [2]:
def missing_data(data):
    total_data = data.isnull().sum()
    precent = (data.isnull().sum()/data.isnull().count()*100)
    tt = pd.concat([total, precent], axis=1, key=["Total", "Percent"])